In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from src.data_handler import DataHandler
from src.analysis_logic import FitMethods

In [ ]:
tip_2R16 = DataHandler(measurement_folder="20221206_FR0612-F2-2R16_PCB_UHVstage_rotated")
saturation = tip_2R16.load_measurements_into_dataclass_list(measurement_str="Saturation")

In [ ]:
sat = saturation[0]

In [ ]:
sat.data.head()

In [ ]:
fig, ax = plt.subplots()

x, y = sat.data["UHV objective power (W)"] * 1e6, sat.data["Avg. counts"] / 1e3
yerr = np.array((sat.data["Upper count errors"] / 1e3, sat.data["Lower count errors"] / 1e3))

ax.errorbar(x, y, yerr=yerr, fmt="o", capsize=5, capthick=1)

fit_x, fit_y, result_hyp = sat.analysis.perform_fit(x, y, fit_function=FitMethods.hyperbolicsaturation)
# ax.plot(fit_x, fit_y, label="Hyperbolic fit")

ax.set_xlabel("Power (μW)")
ax.set_ylabel("Counts (kct/s)")

lin_fit = 14
_, _, result_lin = sat.analysis.perform_fit(x[lin_fit:], y[lin_fit:], fit_function=FitMethods.linear)
ax.plot(x, result_lin.eval(x=x), label="Linear fit")

# ax.axvline(x[lin_fit], linestyle="--", label=f"Crossover at {int(x[lin_fit])} μW")

for idx, val in enumerate(result_lin.eval(x=fit_x) - result_hyp.eval(x=fit_x)):
    if abs(val) < 0.05:
        first_converge = val
        ax.axvline(fit_x[idx], linestyle="--", label=f"Saturation at {int(fit_x[idx])} μW")
        break
        
ax.legend()